In [2]:

API_KEY = 'MLY|7506127936089254|8f1edb43dd2b1fe41af2134a9979f83e'
image_id = '817255772895577'  

## This script downloads a single image, extracts and stores key metadata.

## Dataframes of import

1) df_metadata: captures basic info about the image. image id, dimensions, etc.
2) df_segments: results of the mapillary detections api. This is an 'interim' df as it contains un-normalized coordinates
3) df_detection_coords: normalized detection coordinates, ready to plot

## Storing results in sqlite

I'm not sure what's the smarter thing to store. df_segments with the base64 encoding, or df_detection_coords.
I elected to go for df_detection_coords as the normalization code is lengthy and confusing. The downside is
you have to remember to serialize the resulting list column before storing and then deserialize after loading.

So either way there's a bit of a non-standard process one has to go through when loading from sqlite


# Download image and get geometries

In [3]:
import requests
from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np

import sqlite3
import json

In [49]:
def get_mapillary_image(image_id, api_key):
    def save_image(image, path):
        image.save(path)
        print(f"Image saved to {path}")
    
    headers = {
        'Authorization': f'OAuth {api_key}'
    }
    url = f'https://graph.mapillary.com/{image_id}?fields=thumb_2048_url,captured_at,geometry,height,width'
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Get the image URL and metadata from the response
    data = response.json()
    print(data)
    image_url = data.get('thumb_2048_url')
    image_id = data.get('id')
    captured_at = data.get('captured_at')
    geometry = data.get('geometry')
    detections = data.get('detections')
    original_height = data.get('height')
    original_width = data.get('width')

    if not image_url:
        print("Image URL not found in the response")
        return None, None

    # Download the image
    image_response = requests.get(image_url)
    image_response.raise_for_status()
    image = Image.open(BytesIO(image_response.content))

    save_image_path = f'c:\\temp\\mapillary_{image_id}.jpg'
    save_image(image, save_image_path)
    
    metadata = {
        'image_source' : 'mapillary',
        'image_id': image_id,
        'captured_at_unix': captured_at,
        'lat' : geometry['coordinates'][0],
        'long' : geometry['coordinates'][1],
        'original_height' : original_height,
        'original_width' : original_width,
        'height' : image.height,
        'width' : image.width,
        'image_path_on_disk' : save_image_path
    }
    return image, metadata


def get_mapillary_detections(image_id, api_key):
    headers = {
        'Authorization': f'OAuth {api_key}'
    }
    url = f'https://graph.mapillary.com/{image_id}/detections?fields=image,value,geometry'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    data = response.json()
    return data





In [50]:
image, metadata = get_mapillary_image(image_id, API_KEY)
print("Metadata:", metadata.keys)

{'thumb_2048_url': 'https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6/An9GUcl86gU5sLjydmtSZKuzxMvfiMQlwM2cDPZX9CFv1AWPIIXXRmo0FAJ_Jwet5cNoMTFbr230uwFghPLwnwQzCg-MKB9Fa_gr89SNNWKBJ9PGsA_4JGLN7edZO2I72hT1BMwcJdfweIFUZE4yLw?stp=s2048x1152&ccb=10-5&oh=00_AYAwD0fh9b11Y6GyENfXajsyokQ1K2RZcoqYzaWHajpkZw&oe=66B134A6&_nc_sid=201bca', 'captured_at': 1664012504800, 'geometry': {'type': 'Point', 'coordinates': [98.686175, 3.609915]}, 'height': 2160, 'width': 3840, 'id': '817255772895577'}
Image saved to c:\temp\mapillary_817255772895577.jpg
Metadata: <built-in method keys of dict object at 0x00000284834E6040>


In [51]:
df_metadata = pd.DataFrame.from_dict([metadata])

In [52]:
df_metadata

,image_source,image_id,captured_at_unix,lat,long,original_height,original_width,height,width,image_path_on_disk
0,mapillary,817255772895577,1664012504800,98.686175,3.609915,2160,3840,1152,2048,c:\temp\mapillary_817255772895577.jpg


# Extract Detections and normalize them

In [36]:
import pandas as pd
import base64
import mapbox_vector_tile

In [37]:
def extract_detections(detections):
    payload = []
    for element in detections['data']:
        image_id = element['image']['id']
        detection_label = element['value']
        geometry_base64 = element['geometry']
        detection_id = element['id']
        payload.append([image_id, detection_id, detection_label, geometry_base64])

    df = pd.DataFrame(payload, columns=['image_id','detection_id','detection_label','geometry_base64'])

    return df


#df_segments = extract_detections(detections)

In [38]:
def decode_base64_geometry(base64_string, normalize=False, image_height=None, image_width=None):
    def normalize_detection_geometry(polygon, height, width, extent):
        normalized_polygon = []
        w = []
        y = []
        for xy in polygon:

            # TODO: it's weird that the first element is the width and the second is the height
            # should validate this is correct...
            x_pixel_loc = int((xy[0] / extent) * width)
            y_pixel_loc = int((xy[1] / extent) * height)
    
            w.append(x_pixel_loc)
            y.append(y_pixel_loc)
            normalized_polygon.append([x_pixel_loc, y_pixel_loc])

        return normalized_polygon

    # turn base64 string into a series of x,y coordinates
    decoded_data = base64.decodebytes(base64_string.encode('utf-8'))
    detection_geometry = mapbox_vector_tile.decode(decoded_data)


    # https://www.mapillary.com/developer/api-documentation/
    # mapillary, this needs to be normalized for the image size to get a pixel-by-pixel map
    # I assume this is because the segmentation is done on one size but there are multiple 
    # size options when you download, so I guess this is to scale for the size you're working with.

    extent = detection_geometry['mpy-or']['extent']

    # for any given detection id, there can be 1 or more polygon 'features'
    feature_list = []
    #temp_df = pd.DataFrame(columns=['extent','feature_id','feature_properties','coordinates'])
    for feature in detection_geometry['mpy-or']['features']:
        #print(feature)
        coordinates = feature['geometry']['coordinates']
        feature_id = feature['id']
        feature_properties = feature['properties']

        #each feature can have one or more set of coordinates
        for coord_segment in coordinates:
            #print(coord_segment)

            if normalize == True:
                coord_segment = normalize_detection_geometry(coord_segment, image_height, image_width, extent)

            #temp_df.loc[len(temp_df)] = [extent,feature_id,feature_properties,coord_segment]

            # return a 2d array instead and then explode after the fact
            # makes it easier because the feature doesn't contain the image id
            # which we need to keep data integrity
            feature_list.append([extent,feature_id,feature_properties,coord_segment])

        
    #return feature_list
    return np.array(feature_list, dtype=object)
    


feature_list = decode_base64_geometry('Gp0BeAIKBm1weS1vciiAIBKNARgDCAEihgEJAI4o+gNODRgUMggYFlYkLiwUCAQOGBY2FigHHg4sAAgFHg5AAE4WFA0ADxITKAcIDSIHDA04BzYIDBYUBj6EASIcBBAQBgNCCw4dABMNaQctG20AMQgbKhMQQQdpCDkzDxsDKwcACxUVAB8jYR0THS0NHxsDDxkNAxUTIxsNLQYLGwcADw==', normalize=True, image_height=2160, image_width=3840)


In [39]:
def decode_base64_geometry_fromdf(row, normalize=False, image_height=None, image_width=None):

    base64_string = row['geometry_base64']
    detection_id = row['detection_id']
    detection_label = row['detection_label']

    
    def normalize_detection_geometry(polygon, height, width, extent):
        normalized_polygon = []
        w = []
        y = []
        for xy in polygon:

            # TODO: it's weird that the first element is the width and the second is the height
            # should validate this is correct...
            x_pixel_loc = int((xy[0] / extent) * width)
            y_pixel_loc = int((xy[1] / extent) * height)
    
            w.append(x_pixel_loc)
            y.append(y_pixel_loc)
            normalized_polygon.append([x_pixel_loc, y_pixel_loc])

        return normalized_polygon

    # turn base64 string into a series of x,y coordinates
    decoded_data = base64.decodebytes(base64_string.encode('utf-8'))
    detection_geometry = mapbox_vector_tile.decode(decoded_data)


    # https://www.mapillary.com/developer/api-documentation/
    # mapillary, this needs to be normalized for the image size to get a pixel-by-pixel map
    # I assume this is because the segmentation is done on one size but there are multiple 
    # size options when you download, so I guess this is to scale for the size you're working with.

    extent = detection_geometry['mpy-or']['extent']

    # for any given detection id, there can be 1 or more polygon 'features'
    feature_list = []
    #temp_df = pd.DataFrame(columns=['extent','feature_id','feature_properties','coordinates'])
    for feature in detection_geometry['mpy-or']['features']:
        #print(feature)
        coordinates = feature['geometry']['coordinates']
        feature_id = feature['id']
        feature_properties = feature['properties']

        #each feature can have one or more set of coordinates
        for coord_segment in coordinates:
            #print(coord_segment)

            if normalize == True:
                coord_segment = normalize_detection_geometry(coord_segment, image_height, image_width, extent)

            #temp_df.loc[len(temp_df)] = [extent,feature_id,feature_properties,coord_segment]

            # return a 2d array instead and then explode after the fact
            # makes it easier because the feature doesn't contain the image id
            # which we need to keep data integrity

             	 	 	
            feature_list.append([image_id, detection_id, detection_label, feature_id, extent,feature_properties,coord_segment])

        
    #return feature_list
    return np.array(feature_list, dtype=object)





## Code Execution Begins

In [53]:
detections = get_mapillary_detections(image_id, API_KEY)
df_segments = extract_detections(detections)

In [55]:
#merge w/ metadata so can accesss height/width

df_segments = pd.merge(df_segments,df_metadata, left_on='image_id', right_on='image_id')
df_segments

,image_id,detection_id,detection_label,geometry_base64,image_source,captured_at_unix,lat,long,original_height,original_width,height,width,image_path_on_disk
0,817255772895577,817376652883489,construction--barrier--fence,GjJ4AgoGbXB5LW9yKIAgEiMYAwgBIh0JpgOKIloMFjYOBA...,mapillary,1664012504800,98.686175,3.609915,2160,3840,1152,2048,c:\temp\mapillary_817255772895577.jpg
1,817255772895577,817376656216822,construction--barrier--fence,Gp0BeAIKBm1weS1vciiAIBKNARgDCAEihgEJAI4o+gNODR...,mapillary,1664012504800,98.686175,3.609915,2160,3840,1152,2048,c:\temp\mapillary_817255772895577.jpg
2,817255772895577,817376659550155,construction--barrier--fence,GkJ4AgoGbXB5LW9yKIAgEjMYAwgBIi0JAJIgmgEMAAANIg...,mapillary,1664012504800,98.686175,3.609915,2160,3840,1152,2048,c:\temp\mapillary_817255772895577.jpg
3,817255772895577,817376662883488,construction--barrier--fence,GokBeAIKBm1weS1vciiAIBJ6GAMIASJ0CZ4KjiaiAxYFPA...,mapillary,1664012504800,98.686175,3.609915,2160,3840,1152,2048,c:\temp\mapillary_817255772895577.jpg
4,817255772895577,817376666216821,construction--flat--road,Gip4AgoGbXB5LW9yKIAgEhsYAwgBIhUJ0D2KOjoLGwBBCE...,mapillary,1664012504800,98.686175,3.609915,2160,3840,1152,2048,c:\temp\mapillary_817255772895577.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,817255772895577,817377572883397,object--banner,GlV4AgoGbXB5LW9yKIAgEkYYAwgBIkAJxivSK9oBBAcErQ...,mapillary,1664012504800,98.686175,3.609915,2160,3840,1152,2048,c:\temp\mapillary_817255772895577.jpg
276,817255772895577,817377576216730,object--banner,Gjd4AgoGbXB5LW9yKIAgEigYAwgBIiIJ+CqwKmoETwYpPA...,mapillary,1664012504800,98.686175,3.609915,2160,3840,1152,2048,c:\temp\mapillary_817255772895577.jpg
277,817255772895577,817377579550063,object--support--utility-pole,Gid4AgoGbXB5LW9yKIAgEhgYAwgBIhIJkiOuKSoIiQEMBg...,mapillary,1664012504800,98.686175,3.609915,2160,3840,1152,2048,c:\temp\mapillary_817255772895577.jpg
278,817255772895577,817377582883396,object--vehicle--bus,Glh4AgoGbXB5LW9yKIAgEiQYAwgBIh4JkiPYLVoUBxAIFB...,mapillary,1664012504800,98.686175,3.609915,2160,3840,1152,2048,c:\temp\mapillary_817255772895577.jpg


In [58]:
#get detections and polygons

#arrays = df_segments.apply(decode_base64_geometry_fromdf, normalize=True, image_height=2160, image_width=3840, axis=1)
arrays = df_segments.apply(lambda x: decode_base64_geometry_fromdf(x, normalize=True, image_height=x.height, image_width=x.width), axis=1)


arrays # = np.array(feature_list, dtype=object)
#for arr in arrays:
temp_df = [pd.DataFrame(arr, columns=['image_id', 'detection_id','detection_label','feature_id','extent','properties','coordinates']) for arr in arrays]
df_detection_coords = pd.concat(temp_df, ignore_index=True)

#df_detection_coords

,image_id,detection_id,detection_label,feature_id,extent,properties,coordinates
0,817255772895577,817376652883489,construction--barrier--fence,1,4096,{},"[[105, 538], [108, 535], [122, 533], [123, 531..."
1,817255772895577,817376656216822,construction--barrier--fence,1,4096,{},"[[0, 430], [19, 432], [25, 429], [38, 428], [4..."
2,817255772895577,817376659550155,construction--barrier--fence,1,4096,{},"[[0, 573], [3, 573], [3, 575], [11, 575], [15,..."
3,817255772895577,817376662883488,construction--barrier--fence,1,4096,{},"[[327, 466], [333, 466], [348, 466], [358, 462..."
4,817255772895577,817376666216821,construction--flat--road,1,4096,{},"[[1972, 106], [1969, 110], [1969, 119], [1971,..."
...,...,...,...,...,...,...,...
331,817255772895577,817377576216730,object--banner,1,4096,{},"[[1374, 389], [1375, 400], [1376, 406], [1391,..."
332,817255772895577,817377579550063,object--support--utility-pole,1,4096,{},"[[1124, 407], [1126, 426], [1129, 426], [1129,..."
333,817255772895577,817377582883396,object--vehicle--bus,1,4096,{},"[[1124, 329], [1129, 330], [1133, 329], [1138,..."
334,817255772895577,817377582883396,object--vehicle--bus,2,4096,{},"[[1102, 329], [1117, 330], [1117, 326], [1114,..."


In [42]:
df_detection_coords[df_detection_coords['detection_id'] == '817377582883396']

,image_id,detection_id,detection_label,feature_id,extent,properties,coordinates
333,817255772895577,817377582883396,object--vehicle--bus,1,4096,{},"[[2108, 618], [2117, 620], [2125, 618], [2134,..."
334,817255772895577,817377582883396,object--vehicle--bus,2,4096,{},"[[2066, 618], [2094, 620], [2094, 612], [2088,..."


In [59]:
#save outcomes
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('c://temp//green_repository2.db')

#serialize the coordinates before saving
#df_detection_coords['properties'] = df_detection_coords['properties'].apply(json.dumps)
#df_detection_coords['coordinates'] = df_detection_coords['coordinates'].apply(json.dumps)

#serialize non-scalar objects and save to sqlite
df_metadata = df_metadata.map(json.dumps)
df_metadata.to_sql('image_metadata', con=conn, if_exists='replace', index=False)

#serialize non-scalar objects and save to sqlite
#df_detection_coords['properties'] = df_detection_coords['properties'].apply(json.dumps)
#df_detection_coords['coordinates'] = df_detection_coords['coordinates'].apply(json.dumps)

df_detection_coords = df_detection_coords.map(json.dumps)
df_detection_coords.to_sql('image_detection_coords', con=conn, if_exists='replace', index=False)

#df_x.to_sql('df_foo', con=conn, if_exists='replace', index=False)



# Close the connection
conn.close()


In [44]:
df_detection_coords

,image_id,detection_id,detection_label,feature_id,extent,properties,coordinates
0,"""817255772895577""","""817376652883489""","""construction--barrier--fence""",1,4096,{},"[[197, 1009], [203, 1004], [228, 1000], [230, ..."
1,"""817255772895577""","""817376656216822""","""construction--barrier--fence""",1,4096,{},"[[0, 806], [36, 810], [47, 804], [71, 802], [8..."
2,"""817255772895577""","""817376659550155""","""construction--barrier--fence""",1,4096,{},"[[0, 1075], [5, 1075], [5, 1078], [21, 1078], ..."
3,"""817255772895577""","""817376662883488""","""construction--barrier--fence""",1,4096,{},"[[614, 873], [624, 875], [652, 873], [672, 868..."
4,"""817255772895577""","""817376666216821""","""construction--flat--road""",1,4096,{},"[[3697, 199], [3691, 207], [3691, 224], [3695,..."
...,...,...,...,...,...,...,...
331,"""817255772895577""","""817377576216730""","""object--banner""",1,4096,{},"[[2576, 729], [2578, 750], [2580, 762], [2609,..."
332,"""817255772895577""","""817377579550063""","""object--support--utility-pole""",1,4096,{},"[[2108, 764], [2112, 800], [2117, 798], [2117,..."
333,"""817255772895577""","""817377582883396""","""object--vehicle--bus""",1,4096,{},"[[2108, 618], [2117, 620], [2125, 618], [2134,..."
334,"""817255772895577""","""817377582883396""","""object--vehicle--bus""",2,4096,{},"[[2066, 618], [2094, 620], [2094, 612], [2088,..."


## end

In [19]:
# iterrate over the segments and build another df with all the normalized polygons

df_normalized_segments = pd.DataFrame(columns = ['extent','feature_id','feature_properties','coordinates'])

s64s = df_segments['geometry_base64'].tolist()
for g in s64s:
    temp_df = decode_base64_geometry(g, normalize=True, image_height=2160, image_width=3840 )
    df_normalized_segments = pd.concat([df_normalized_segments,temp_df])
    #df_normalized_segments.loc[len(df_normalized_segments)] = [temp_df]
    #print(geo_json)
    #print("\n\n")
    #print(f"{len(geo_json['mpy-or']['features'])} -- {geo_json['mpy-or']['type']}")

In [20]:
df_normalized_segments

,extent,feature_id,feature_properties,coordinates
0,4096,1,{},"[[197, 1009], [203, 1004], [228, 1000], [230, ..."
0,4096,1,{},"[[0, 806], [36, 810], [47, 804], [71, 802], [8..."
0,4096,1,{},"[[0, 1075], [5, 1075], [5, 1078], [21, 1078], ..."
0,4096,1,{},"[[614, 873], [624, 875], [652, 873], [672, 868..."
0,4096,1,{},"[[3697, 199], [3691, 207], [3691, 224], [3695,..."
...,...,...,...,...
0,4096,1,{},"[[2576, 729], [2578, 750], [2580, 762], [2609,..."
0,4096,1,{},"[[2108, 764], [2112, 800], [2117, 798], [2117,..."
0,4096,1,{},"[[2108, 618], [2117, 620], [2125, 618], [2134,..."
1,4096,2,{},"[[2066, 618], [2094, 620], [2094, 612], [2088,..."


In [ ]:
{'image': {'geometry': {'type': 'Point', 'coordinates': [98.686175, 3.609915]},
  'id': '817255772895577'},
 'value': 'construction--barrier--fence',
 'geometry': 'GjJ4AgoGbXB5LW9yKIAgEiMYAwgBIh0JpgOKIloMFjYOBA4gEAMOJwANFR8HCxUXDQgNDw==',
 'id': '817376652883489'}